# Process Data
---

Clean the CSVs that were provided.

1. Clean column names
2. Remove unneeded columns (e.g. 6 or 12-months post treatment)
3. Remove dropout patients
4. Set missing values to NaN

## Import Packages & Setup Variables
---

In [43]:
import pandas as pd
import numpy as np

import os
import re

from pathlib import Path
from collections import OrderedDict

from sklearn.preprocessing import StandardScaler

from pandas_profiling import ProfileReport

## Import + Clean Data
---

In [7]:
def fix_column_names(df):
    """
    Clean the column names, similar to clean_names in tidyverse janitor
      - CamelCase to snake_case
      - All characters to lower case
      - Remove special characters
      - Replace whitespace with underscore (_)
    
    Returns
    ---
    pd.DataFrame
    """
    df = df.copy()
    
    # column names to list
    column_list = [*df.columns]

    # 3+ capital letters to lowercase (e.g. API)
    column_list = \
      [re.sub(r"[A-Z]{3,}", lambda m: m.group(0).lower(), x) for x in column_list]

    # # convert to snake_case
    column_list = [re.sub(r'(?<!^)(?=[A-Z])', '_', x) for x in column_list]
    
    # # remove special characters
    column_list = [re.sub(r"[^\w\s]", '', x) for x in column_list]

    # # whitespace to underscore
    column_list = [re.sub(r"\s+", '_', x) for x in column_list]

    # # Fix double underscores
    column_list = [re.sub(r"(_{2,})", "_", x) for x in column_list]
    
    # assign to dataframe
    df.columns = column_list

    # lower case
    df = df.rename(str.lower, axis='columns')
    
    return df

In [33]:
df = pd.read_csv("../data/chronic_pain_reduced.csv")
df = fix_column_names(df)

## Drop Columns + Dropout Patients!
---
* Drop columns that we will not use (e.g. post-6, post-12 month data)
* Remove patients that were dropouts (10 total)
* Set Missing values to NA

In [34]:
# Drop records containing values
# Using an OrderedDict in case there is a specific reason to remove something first
record_drop = OrderedDict({
    'status': ['Dropout Post-Randomization']
})

In [35]:
# Replace missing value placeholders with np.nan()
missing_values = {
    'api_tot_recode0_4_post': 9999,
    'csigimn2019_post': ' ',
    'cprinterf8_post': 9999,
    'cprint8_tscore_post': 9999,
    'pbq3_txfap': ' ',
    'pbq6_txfap': ' ',
    'pbq13_txfap': ' ',
    'pbq14_txfap': ' ',
    'prsleep8_txfap': ' '
}

In [36]:
# Drop columns
cols_to_drop = [
    'redcap_survey_identifier',                 # unique ID for each survey participant
    'status',                                   # Enrolled. Already dropped the dropouts
    'api_tot_recode0_4_mid',
    'api_tot_recode0_4_6mo',
    'api_tot_recode0_4_12mo',
    'csigimn2019_mid',
    'csigimn2019_6mo',
    'csigimn2019_12mo',
    'cprinterf8_mid',
    'cprint8_tscore_mid',
    'cprinterf8_6mo',
    'cprint8_tscore_6mo',
    'cprinterf8_12mo',
    'cprint8_tscore_12mo',
    'dose',
    'teen_module'
]

In [37]:
## Drop values from DF
for key, value in record_drop.items():
    df = df[~df[key].isin(value)]

# Drop them columns
df = df.drop(cols_to_drop, axis=1)

In [38]:
# Missing values to NaN
for key, value in missing_values.items():
    df.loc[df[key] == value, key] = np.nan

# Quick remedy for now --  Replace all empty strings with nan
df = df.replace(' ', np.nan, regex=True)
df = df.replace('', np.nan, regex=True)

In [39]:
# Drop those where we have a missing outcome variable - not much we can do there
# Removes 43 records (16%)
df = df.dropna(subset = [
    'api_tot_recode0_4_post', 'csigimn2019_post', 'cprinterf8_post', 'cprint8_tscore_post',
    'api_tot_recode0_4_base', 'csigimn2019_base', 'cprinterf8_base', 'cprint8_tscore_base'
])

### Write CSV #1 -- Includes Missing Data

Can be used to test imputing missing values later on

In [40]:
df.to_csv("../data/cleaned/not_scaled_with_missing.csv", index=False, na_rep='NULL')

### Write CSV #2 -- Excludes Missing Data

In [41]:
# Drop NA for remaining data -- Need to think of a strategy
# Go from 268 to 225 records (43 records 16% loss)
# 19 from Study Group 2, 14 from Study Group 1

# IF WE DO THIS TO ALL DATA COLUMNS WE END UP WITH ONLY 184 RECORDS (-84, 31%)
df_no_missing = df.dropna(axis=0).copy()

In [42]:
df_no_missing.to_csv("../data/cleaned/not_scaled_with_missing.csv", index=False, na_rep='NULL')

## Rescale Variables
---

We want all of our features to be aligned with how they scale (low to high). In the code below we assume any feature not listed is already scaled low-high.

Repeat the same code block twice -- one that includes missing data, one that does not have missing data

In [49]:
# Reverse the scaling based on min/max values provided (don't want to infer directly from data)
# Assuming all values are integers, for now
# Notes here: https://docs.google.com/spreadsheets/d/1wSv6wUGWKldj-Rhy_BoqFUwiF6j74xf96LVe9aiPM1Q/edit#gid=0
rescale_columns = {
    "pbq1_txfap": [0, 4],
    "pbq3_txfap": [0, 4],
    "pbq6_txfap": [0, 4],
    "pbq7_txfap": [0, 4],
    "pbq8_txfap": [0, 4],
    "pbq11_txfap": [0, 4],
    "pbq13_txfap": [0, 4],
    "pbq14_txfap": [0, 4],
    "pbq15_txfap": [0, 4],
    "pbq16_txfap": [0, 4],
    "pbq17_txfap": [0, 4],
    "pbq18_txfap": [0, 4],
    "prsleep2_txfap": [0, 4],
    "prsleep3_txfap": [0, 4],
    "prsleep8_txfap": [1, 5],
}

In [50]:
# DF - WITH MISSING DATA
# Loop through each item in the column list
previous_scale = ''

for col, question_scale in rescale_columns.items():
    
    # Error handling
    if col not in df.columns:
        print('NEED TO RAISE ERROR!')
    
    # Skip if the same values as previous column
    if previous_scale != question_scale:

        # Create reversed question scale (inclusive)
        quest_scale_reverse = [*range(question_scale[0], question_scale[1] + 1)][::-1]

        # prepend 0s if starting at non-zero
        if question_scale[0] != 0:
            quest_scale_reverse = [0] * question_scale[0] + quest_scale_reverse

        print(quest_scale_reverse)

    else:
        print(f'Didn\'t update rescaler --- {col}')
    
    # rescale all data points in that 
    df[col] = df[col].map(lambda x: quest_scale_reverse[int(x)], na_action='ignore')
    
    # Last thing to do
    previous_scale = question_scale

df.to_csv("../data/cleaned/scaled_with_missing.csv", index=False)

[4, 3, 2, 1, 0]
Didn't update rescaler --- pbq3_txfap
Didn't update rescaler --- pbq6_txfap
Didn't update rescaler --- pbq7_txfap
Didn't update rescaler --- pbq8_txfap
Didn't update rescaler --- pbq11_txfap
Didn't update rescaler --- pbq13_txfap
Didn't update rescaler --- pbq14_txfap
Didn't update rescaler --- pbq15_txfap
Didn't update rescaler --- pbq16_txfap
Didn't update rescaler --- pbq17_txfap
Didn't update rescaler --- pbq18_txfap
Didn't update rescaler --- prsleep2_txfap
Didn't update rescaler --- prsleep3_txfap
[0, 5, 4, 3, 2, 1]


In [51]:
# DF - NO MISSING DATA
# Loop through each item in the column list
previous_scale = ''

for col, question_scale in rescale_columns.items():
    
    # Error handling
    if col not in df_no_missing.columns:
        print('NEED TO RAISE ERROR!')
    
    # Skip if the same values as previous column
    if previous_scale != question_scale:

        # Create reversed question scale (inclusive)
        quest_scale_reverse = [*range(question_scale[0], question_scale[1] + 1)][::-1]

        # prepend 0s if starting at non-zero
        if question_scale[0] != 0:
            quest_scale_reverse = [0] * question_scale[0] + quest_scale_reverse

        print(quest_scale_reverse)

    else:
        print(f'Didn\'t update rescaler --- {col}')
    
    # rescale all data points in that 
    df_no_missing[col] = df_no_missing[col].map(lambda x: quest_scale_reverse[int(x)], na_action='ignore')
    
    # Last thing to do
    previous_scale = question_scale

df_no_missing.to_csv("../data/cleaned/scaled_no_missing.csv", index=False)

[4, 3, 2, 1, 0]
Didn't update rescaler --- pbq3_txfap
Didn't update rescaler --- pbq6_txfap
Didn't update rescaler --- pbq7_txfap
Didn't update rescaler --- pbq8_txfap
Didn't update rescaler --- pbq11_txfap
Didn't update rescaler --- pbq13_txfap
Didn't update rescaler --- pbq14_txfap
Didn't update rescaler --- pbq15_txfap
Didn't update rescaler --- pbq16_txfap
Didn't update rescaler --- pbq17_txfap
Didn't update rescaler --- pbq18_txfap
Didn't update rescaler --- prsleep2_txfap
Didn't update rescaler --- prsleep3_txfap
[0, 5, 4, 3, 2, 1]


## Pandas Profiling
---

In [59]:
profile = ProfileReport(df_no_missing, title='Pandas Profiling Report')

In [60]:
profile.to_file("../pandas_profiling_report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 23.52it/s]
